intialize a Graph object

In [2]:
import rdflib as rdf
import rdflib.namespace

In [3]:
g = rdf.Graph()

## anytime crepes

<https://www.food.com/recipe/327593>

  * id: 327593
  * minutes: 8
  * ingredients: "['egg', 'milk', 'whole wheat flour']"


load CSV data which is a subset from <https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions/metadata>

In [5]:
import pandas as pd

df = pd.read_csv("recipes.csv")
df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,anytime crepes,327593,8,955857,2008-09-27,"['15-minutes-or-less', 'time-to-make', 'course...","[149.6, 14.0, 0.0, 5.0, 20.0, 21.0, 2.0]",12,"['mix all ingredients in a blender , or mix re...","from my friend linda, this is an oh-so-easy-an...","['egg', 'milk', 'whole wheat flour']",3


In [6]:
recipe_row = df[df["name"] == "anytime crepes"].iloc[0]
recipe_row

name                                                 anytime crepes
id                                                           327593
minutes                                                           8
contributor_id                                               955857
submitted                                                2008-09-27
tags              ['15-minutes-or-less', 'time-to-make', 'course...
nutrition                  [149.6, 14.0, 0.0, 5.0, 20.0, 21.0, 2.0]
n_steps                                                          12
steps             ['mix all ingredients in a blender , or mix re...
description       from my friend linda, this is an oh-so-easy-an...
ingredients                    ['egg', 'milk', 'whole wheat flour']
n_ingredients                                                     3
Name: 0, dtype: object

In [7]:
ingredients = eval(recipe_row["ingredients"])
ingredients

['egg', 'milk', 'whole wheat flour']

To represent a recipe, we will use two *controlled vocabularies*:

  * <http://purl.org/heals/food/>
  * <http://purl.org/heals/ingredient/>

So then the ingredients list becomes linked to:

  * `ingredient:ChickenEgg`
  * `ingredient:CowMilk`
  * `ingredient:WholeWheatFlour`

    <owl:NamedIndividual rdf:about="&ind;BananaBlueberryAlmondFlourMuffin">
        <rdf:type rdf:resource="&wtm;Recipe"/>
        <wtm:hasIngredient rdf:resource="&ind;AlmondMeal"/>
        <wtm:hasIngredient rdf:resource="&ind;AppleCiderVinegar"/>
        <wtm:hasIngredient rdf:resource="&ind;BakingSoda"/>
        <wtm:hasIngredient rdf:resource="&ind;Banana"/>
        <wtm:hasIngredient rdf:resource="&ind;Blueberry"/>
        <wtm:hasIngredient rdf:resource="&ind;ChickenEgg"/>
        <wtm:hasIngredient rdf:resource="&ind;Honey"/>
        <wtm:isRecommendedForCourse rdf:resource="&wtm;Dessert"/>
        <wtm:isRecommendedForMeal rdf:resource="&wtm;Breakfast"/>
        <wtm:isRecommendedForMeal rdf:resource="&wtm;Snack"/>
        <wtm:hasCookTime rdf:datatype="&xsd;integer">60</wtm:hasCookTime>
        <wtm:hasCookingTemperature rdf:datatype="&xsd;integer">350</wtm:hasCookingTemperature>
        <wtm:serves rdf:datatype="&xsd;integer">4</wtm:serves>
        <dct:source>Allrecipes.com, "Banana Blueberry Almond Flour Muffins ," 2010. [Online]. Available: https://www.allrecipes.com/recipe/238012/banana-blueberry-almond-flour-muffins-gluten-free/?internalSource=hub%20recipe&amp;referringContentType=Search. [Accessed Nov.9, 2018]</dct:source>
        <rdfs:label>banana blueberry almond flour muffin</rdfs:label>
        <skos:definition>a banana blueberry muffin made with almond flour</skos:definition>
        <skos:scopeNote>recipe</skos:scopeNote>
        <prov:wasDerivedFrom>https://www.allrecipes.com/recipe/238012/banana-blueberry-almond-flour-muffins-gluten-free/?internalSource=hub%20recipe&amp;referringContentType=Search</prov:wasDerivedFrom>
    </owl:NamedIndividual>

```
:x rdf:type wtm:Recipe ,
  wtm:hasIngredient ingredient:ChickenEgg,
  wtm:hasIngredient ingredient:CowMilk,
  wtm:hasIngredient ingredient:WholeWheatFlour,
  wtm:hasCookTime 8,
  skos:definition "anytime crepes",
  dct:source <https://www.food.com/recipe/327593>
  ;
```

In [8]:
uri = "https://www.food.com/recipe/327593"
s = rdf.URIRef(uri)
s

rdflib.term.URIRef('https://www.food.com/recipe/327593')

In [9]:
from rdflib.namespace import RDF

p = RDF.type
p

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')

In [10]:
nm = rdflib.namespace.NamespaceManager(g)

In [11]:
uri = "http://purl.org/heals/food/"
ns_wtm = rdf.Namespace(uri)

prefix = "wtm"
nm.bind(prefix, ns_wtm)

In [12]:
o = ns_wtm.Recipe
o

rdflib.term.URIRef('http://purl.org/heals/food/Recipe')

In [13]:
g.add((s, p, o,))
g

<Graph identifier=N469c9aa9c1364fa0b9b68d2678aaac48 (<class 'rdflib.graph.Graph'>)>

let's add more...

In [14]:
p = ns_wtm.hasCookTime
p

rdflib.term.URIRef('http://purl.org/heals/food/hasCookTime')

In [15]:
from rdflib.namespace import XSD

o = rdf.Literal("8", datatype=XSD.integer)
o

rdflib.term.Literal('8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))

In [16]:
g.add((s, p, o,))

now let's add the ingredients

In [17]:
p = ns_wtm.hasIngredient
p

rdflib.term.URIRef('http://purl.org/heals/food/hasIngredient')

In [18]:
uri = "http://purl.org/heals/ingredient/"
ns_ind = rdf.Namespace(uri)

prefix = "ind"
nm.bind(prefix, ns_ind)

In [19]:
o = ns_ind.ChickenEgg
o

rdflib.term.URIRef('http://purl.org/heals/ingredient/ChickenEgg')

In [20]:
g.add((s, p, o,))

In [21]:
g.add((s, p, ns_ind.CowMilk,))
g.add((s, p, ns_ind.WholeWheatFlour,))

In [22]:
for s, p, o in g:
    print(s, p, o)

https://www.food.com/recipe/327593 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://purl.org/heals/food/Recipe
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasCookTime 8
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/WholeWheatFlour
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/CowMilk
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/ChickenEgg


show a serialization to `n3` triples, also known as `ttl` or "turtle" format

In [23]:
s = g.serialize(format="n3")
print(s.decode("utf8"))

@prefix ind: <http://purl.org/heals/ingredient/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix wtm: <http://purl.org/heals/food/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://www.food.com/recipe/327593> a wtm:Recipe ;
    wtm:hasCookTime 8 ;
    wtm:hasIngredient ind:ChickenEgg,
        ind:CowMilk,
        ind:WholeWheatFlour .




serialization as `n3` triples to a file

In [24]:
g.serialize(destination="tmp.ttl", format="n3", encoding="utf-8")

try taking a look at the `tmp.ttl` file, which should be the same as the serialization above

In [26]:
data = g.serialize(
    format="json-ld",
    indent=2,
    encoding="utf-8",
    )

with open("tmp.jsonld", "wb") as f:
    f.write(data)

try taking a look at the `tmp.jsonld` file

In [28]:
context = {
    "@language": "en",
    "wtm": "http://purl.org/heals/food/",
    "ind": "http://purl.org/heals/ingredient/",
    }

In [29]:
context

{'@language': 'en',
 'wtm': 'http://purl.org/heals/food/',
 'ind': 'http://purl.org/heals/ingredient/'}

In [30]:
data = g.serialize(
    format="json-ld",
    context=context,
    indent=2,
    encoding="utf-8",
    )

with open("tmp.jsonld", "wb") as f:
    f.write(data)